![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Financial Word and Sentence Embeddings

# Finance Word and Sentence Embeddings visualization using PCA (Principal Component Analysis)

Modern NLP models work with a numerical representation of texts and their menaning. For token classification problems (inferring a class for a token, for example Name Entity Recognition) Word Embeddings are required. For sentences, paragraph, document classification - we use Sentence Embeddings.

In this notebook, we got token embeddings using Spark NLP Finance Word Embeddings(**bert_embeddings_sec_bert_base**) and using these token embeddings we got sentence embeddings by sparknlp annotator SentenceEmbeddings to get those numerical representations of the semantics of the texts. The result is a 768 embeddings matrix, impossible to process by the human eye.

There are many techniques we can use to visualize those embeddings. We are using one of them - Principal Component Analysis, a dimensionality reduction process, carried out by Spark MLLib. Both embeddings have 768 dimensions, so we will reduced this dimensions from **768** to **3** (X, Y, Z) and will use a color for the word / sentence legend.

# Installation

In [0]:
from johnsnowlabs import * 
import pandas as pd

# Get sample text

In [0]:
# Downloading sample datasets.
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/Finance/data/finance_pca_samples.csv
dbutils.fs.cp("file:/databricks/driver/finance_pca_samples.csv", "dbfs:/")

Out[4]: True

In [0]:
import pandas as pd

df = pd.read_csv("finance_pca_samples.csv")

In [0]:
# Create spark dataframe
sdf = spark.createDataFrame(df)
sdf.show()

+--------------------+----------------+
 text| label|
+--------------------+----------------+
I called Huntingt...| Accounts|
I opened an citi ...| Accounts|
I have been a lon...| Credit Cards|
My credit limit w...| Credit Cards|
I am filing this ...|Credit Reporting|
The Credit Bureau...|Credit Reporting|
I noticed an arti...| Debt Collection|
A bank account wa...| Debt Collection|
I was contacted v...| Loans|
My husband recent...| Loans|
I wire transfered...| Money Transfers|
PayPal holds fund...| Money Transfers|
We have requested...| Mortgage|
I filled out a co...| Mortgage|
+--------------------+----------------+

# Pipeline with Spark NLP and Spark MLLIB

In [0]:
# We defined a generic pipeline for word and sentence embeddings

def generic_pipeline():
  document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

  tokenizer = nlp.Tokenizer()\
      .setInputCols("document")\
      .setOutputCol("token")

  word_embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en")\
      .setInputCols(["document", "token"])\
      .setOutputCol("word_embeddings")

  pipeline = nlp.Pipeline(stages = [
      document_assembler,
      tokenizer,
      word_embeddings
  ])

  return pipeline



## Sentence Embeddings

In [0]:
embeddings_sentence = nlp.SentenceEmbeddings()\
    .setInputCols(["document", "word_embeddings"])\
    .setOutputCol("sentence_embeddings")\
    .setPoolingStrategy("AVERAGE")
# We used sparknlp SentenceEmbeddings anootator to get each sentence embeddings from token embeddings

# Custom transform to retrieve the numerical embeddings from Spark NLP and pass it to Spark MLLib

In [0]:
# This class extracts the embeddings from the Spark NLP Annotation object
# from pyspark import ml as ML
## < 2.0 -> pyspark.ml.util.keyword_only
from pyspark.ml import Transformer
from pyspark import ml as ML
from pyspark import keyword_only 

class EmbeddingsUDF(
    Transformer, ML.param.shared.HasInputCol, ML.param.shared.HasOutputCol,
    ML.util.DefaultParamsReadable, ML.util.DefaultParamsWritable
):
    @keyword_only
    def __init__(self):
        super(EmbeddingsUDF, self).__init__()

        def _sum(r):
            result = 0.0
            for e in r:
                result += e
            return result

        self.udfs = {
            'convertToVectorUDF': F.udf(lambda vs: ML.linalg.Vectors.dense(vs), ML.linalg.VectorUDT()),
            'sumUDF': F.udf(lambda r: _sum(r), T.FloatType())
        }

    def _transform(self, dataset):

        results = dataset.select(
            "*", F.explode("sentence_embeddings.embeddings").alias("embeddings")
        )
        results = results.withColumn(
            "features",
            self.udfs['convertToVectorUDF'](F.col("embeddings"))
        )
        results = results.withColumn(
            "emb_sum",
            self.udfs['sumUDF'](F.col("embeddings"))
        )
        # Remove those with embeddings all zeroes (so we can calculate cosine distance)
        results = results.where(F.col("emb_sum")!=0.0)

        return results

In [0]:
from pyspark.sql import functions as F
import pyspark.sql.types as T

embeddings_for_pca = EmbeddingsUDF()

In [0]:
DIMENSIONS  = 3

In [0]:
pca = ML.feature.PCA(k=DIMENSIONS, inputCol="features", outputCol="pca_features")

### Full Spark NLP + Spark MLLib pipeline

In [0]:
# We did all process in one pipeline
pipeline = nlp.Pipeline().setStages([generic_pipeline(), embeddings_sentence, embeddings_for_pca, pca])

bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]

In [0]:
model = pipeline.fit(sdf)

In [0]:
result = model.transform(sdf)

In [0]:
result.select('pca_features', 'label').show(truncate=False)

+-------------------------------------------------------------+----------------+
pca_features |label |
+-------------------------------------------------------------+----------------+
[3.395761733456288,-1.0603583667210579,-1.5687931246986362] |Accounts |
[2.366080448286008,0.8591978691981672,-0.8066172037253643] |Accounts |
[0.6867676011553301,1.4823995675537482,0.006590845227330496] |Credit Cards |
[-0.28834654292887507,1.0031584374166118,-0.7963799783034821]|Credit Cards |
[-0.5037846809116963,-1.3771537751758807,0.4449728258742432] |Credit Reporting|
[1.0397520771220115,-1.7194098205845816,1.8539401317975641] |Credit Reporting|
[2.7731648234189037,1.1680314504169131,1.3949444184084288] |Debt Collection |
[-0.45951420852073444,0.8339738969601689,0.5051705970262341] |Debt Collection |
[0.2703033212234329,1.1069461667059015,-0.4247561222443627] |Loans |
[0.8662474660433654,1.1435305015895525,0.8703545815541563] |Loans |
[-0.7581009820669184,0.6312481909821599,0.6829063142780839] |Money Transfers |
[0.3855731881553479,-1.4420949248493304,-0.1982527248453405] |Money Transfers |
[2.4568999166846672,0.3302625717690851,1.2981715856527762] |Mortgage |
[2.355323630582012,0.8329546675421802,1.3904067418685468] |Mortgage |
+-------------------------------------------------------------+----------------+

In [0]:
df = result.select('pca_features', 'label').toPandas()

df
# As you see, dimension values are inside a list

Out[23]:

,pca_features,label
0,"[3.395761733456288, -1.0603583667210579, -1.56...",Accounts
1,"[2.366080448286008, 0.8591978691981672, -0.806...",Accounts
2,"[0.6867676011553301, 1.4823995675537482, 0.006...",Credit Cards
3,"[-0.28834654292887507, 1.0031584374166118, -0....",Credit Cards
4,"[-0.5037846809116963, -1.3771537751758807, 0.4...",Credit Reporting
5,"[1.0397520771220115, -1.7194098205845816, 1.85...",Credit Reporting
6,"[2.7731648234189037, 1.1680314504169131, 1.394...",Debt Collection
7,"[-0.45951420852073444, 0.8339738969601689, 0.5...",Debt Collection
8,"[0.2703033212234329, 1.1069461667059015, -0.42...",Loans
9,"[0.8662474660433654, 1.1435305015895525, 0.870...",Loans


In [0]:
# We extract the dimension values out off the list

df["x"] = df["pca_features"].apply(lambda x: x[0])

df["y"] = df["pca_features"].apply(lambda x: x[1])

df["z"] = df["pca_features"].apply(lambda x: x[2])

df = df[["x", "y", "z", "label"]]

df

Out[24]:

,x,y,z,label
0,3.395762,-1.060358,-1.568793,Accounts
1,2.366080,0.859198,-0.806617,Accounts
2,0.686768,1.482400,0.006591,Credit Cards
3,-0.288347,1.003158,-0.796380,Credit Cards
4,-0.503785,-1.377154,0.444973,Credit Reporting
5,1.039752,-1.719410,1.853940,Credit Reporting
6,2.773165,1.168031,1.394944,Debt Collection
7,-0.459514,0.833974,0.505171,Debt Collection
8,0.270303,1.106946,-0.424756,Loans
9,0.866247,1.143531,0.870355,Loans


In [0]:
import plotly.express as px

fig = px.scatter_3d(df, x = 'x', y = 'y', z = 'z', color = 'label', width=800, height=600)

fig.show()

### Word Embeddings

We can also visualize the semantics of words, instead of full texts, by using Word Embeddings. We will add a Tokenizer and a WordEmbeddings model to get those embeddings, and them apply PCA as before. Firstly we splitted the pipeline in two to get all token embeddings

In [0]:
model = generic_pipeline().fit(sdf)

bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]

In [0]:
result = model.transform(sdf)

In [0]:
from pyspark.sql import functions as F

result_df = result.select("label", F.explode(F.arrays_zip(result.token.result, result.word_embeddings.embeddings)).alias("cols"))\
      .select(F.expr("cols['0']").alias("token"),
              "label",
              F.expr("cols['1']").alias("embeddings"))

result_df.show(truncate = 80)


+----------+--------+--------------------------------------------------------------------------------+
 token| label| embeddings|
+----------+--------+--------------------------------------------------------------------------------+
 I|Accounts|[-0.29679197, 0.8095251, 0.026026228, 0.08434205, 0.74346256, -0.026947215, -...|
 called|Accounts|[0.28905842, -0.29229757, -0.4299038, -0.38334602, 0.026178751, -0.12728381, ...|
Huntington|Accounts|[0.20684525, -0.010130528, -0.25902477, -0.37558505, 0.45792055, 0.31149128, ...|
 Bank|Accounts|[-0.034711145, 0.46047518, -0.6221117, -0.011170335, 0.2938506, 0.3134117, -0...|
 to|Accounts|[-0.40457875, -0.37686485, -0.08015403, -0.58909625, -0.3385651, -0.3932127, ...|
 close|Accounts|[0.35089368, 0.95684683, 0.8632827, -0.4334406, 0.11386813, -0.48837754, -0.8...|
 my|Accounts|[-0.3659187, 0.26556098, -0.3249508, -0.5081892, -0.39623892, -0.6334721, -0....|
 account|Accounts|[0.0046396814, 0.53401303, 0.7756802, 0.23316619, -0.43037707, -0.29379046, -...|
 ,|Accounts|[0.17874274, -0.026907708, 0.1949839, -0.792961, -0.26044437, -0.3964326, -0....|
 and|Accounts|[0.50113493, 0.66375434, 0.15587755, -0.7952295, -0.8198415, -0.24028674, -0....|
 they|Accounts|[-0.21890028, 0.17353031, -0.3897712, -0.42199916, -0.6609, -0.6682682, -0.33...|
 refused|Accounts|[-0.7153429, 0.40928861, -0.5824087, 0.2768948, -0.74408066, -0.0168421, -0.3...|
 to|Accounts|[-0.062417313, -0.30230477, 0.17689252, -0.36983997, 0.2230865, -0.20912707, ...|
 close|Accounts|[0.49901596, 0.933635, 0.89050436, -0.20053637, 0.47381905, -0.24397752, -0.7...|
 my|Accounts|[-0.11864838, 0.06864378, -0.4704897, -0.33866665, -0.14482069, -0.5999235, -...|
 account|Accounts|[-0.045061324, 0.5524493, 0.9458424, 0.3263071, -0.26439258, -0.14597295, -0....|
 over|Accounts|[0.19230434, -0.47188896, 0.3358269, 0.008950591, 0.34794202, 0.10783947, -0....|
 the|Accounts|[-0.44176185, -0.1791171, -0.96231747, 0.097165085, 0.19224177, 0.15848777, 0...|
 phone|Accounts|[-0.4497391, -0.9114664, -0.06911227, -0.18094975, 0.108374484, -0.8229779, -...|
 .|Accounts|[0.050267164, 0.32013276, 0.22356187, -0.6540267, 0.48769468, -0.8169061, -0....|
+----------+--------+--------------------------------------------------------------------------------+
only showing top 20 rows

In [0]:
# Here we defined inheritance class from that defined previously EmbeddingsUDF class
class WordEmbeddingsUDF(EmbeddingsUDF):    
    def _transform(self, dataset):
        
        results = dataset.select('token', 'label', 'embeddings') # We changed this line because our embedding cloumn is already exploded

        results = results.withColumn(
            "features",
            self.udfs['convertToVectorUDF'](F.col("embeddings"))
        )
        results = results.withColumn(
            "emb_sum",
            self.udfs['sumUDF'](F.col("embeddings"))
        )
        # Remove those with embeddings all zeroes (so we can calculate cosine distance)
        results = results.where(F.col("emb_sum")!=0.0)

        return results

In [0]:
embeddings_for_pca = WordEmbeddingsUDF()

In [0]:
DIMENSIONS  = 3

In [0]:
pca = ML.feature.PCA(k=DIMENSIONS, inputCol="features", outputCol="pca_features")

### Full Spark NLP + Spark MLLib pipeline

In [0]:
# We run the second part of the pipeline. Here 768 dimensions is reduced to 3 dimensions

pipeline = nlp.Pipeline().setStages([embeddings_for_pca, pca])


In [0]:
model = pipeline.fit(result_df)

In [0]:
result = model.transform(result_df)

In [0]:
result.select("token", "label", "pca_features").show(truncate = 60)

+----------+--------+------------------------------------------------------------+
 token| label| pca_features|
+----------+--------+------------------------------------------------------------+
 I|Accounts| [9.85046796699116,0.021824760616261294,1.7128893179625404]|
 called|Accounts| [0.5703253366247248,0.34666249136598587,-2.867726953838327]|
Huntington|Accounts| [8.635446663422016,0.8802328948982534,-0.8417114914739703]|
 Bank|Accounts| [9.39106134939763,0.4506675629754985,-1.2157421173156366]|
 to|Accounts| [-2.093784238264132,-1.1261847621412933,4.473376741277198]|
 close|Accounts|[-2.8977636323317886,-0.16330212547120818,2.6316561678661...|
 my|Accounts| [3.54223999495112,-2.721488198385069,2.8479003615325404]|
 account|Accounts| [-1.253327572579749,0.00648285380304152,1.9023200680945112]|
 ,|Accounts| [-1.3713437212054282,0.16043920722480115,2.236148638440996]|
 and|Accounts|[0.25747849570313136,-0.39881826391811315,4.898581479723758]|
 they|Accounts| [2.6491819279224904,-2.0965519734873626,3.004774210438144]|
 refused|Accounts| [-1.4478400703353695,-3.120719874833148,1.6237177112842545]|
 to|Accounts| [-3.4761363553968376,-0.9551206983865539,5.927978332349893]|
 close|Accounts|[-3.220328986048479,-0.09380215236055814,2.5022196372619017]|
 my|Accounts| [3.0773975431424954,-2.7323425884854555,3.4639012382706853]|
 account|Accounts|[-1.7590960507902684,0.049204285410867535,2.0855736923233...|
 over|Accounts| [-0.7852920276768591,-1.083722410035382,2.775248550136555]|
 the|Accounts| [-1.9152536543136534,-0.5845541159760957,5.449712012235361]|
 phone|Accounts|[-3.7960458965585078,-2.5272429402509076,1.5636350468685576]|
 .|Accounts|[-0.0421216550835699,-0.5960689223393846,0.48708454480074...|
+----------+--------+------------------------------------------------------------+
only showing top 20 rows

In [0]:
df = result.select('token', 'label', 'pca_features').toPandas()

df

Out[37]:

,token,label,pca_features
0,I,Accounts,"[9.85046796699116, 0.021824760616261294, 1.712..."
1,called,Accounts,"[0.5703253366247248, 0.34666249136598587, -2.8..."
2,Huntington,Accounts,"[8.635446663422016, 0.8802328948982534, -0.841..."
3,Bank,Accounts,"[9.39106134939763, 0.4506675629754985, -1.2157..."
4,to,Accounts,"[-2.093784238264132, -1.1261847621412933, 4.47..."
...,...,...,...
1364,the,Mortgage,"[0.20783244995060898, 1.2121737442831522, 2.34..."
1365,company,Mortgage,"[0.9758776803817245, 1.1525679357974887, 1.548..."
1366,never,Mortgage,"[-0.009449005811853118, -1.3605046537433145, -..."
1367,responds,Mortgage,"[-1.3105358968915206, -0.39519741635318867, -1..."


In [0]:
df["x"] = df["pca_features"].apply(lambda x: x[0])

df["y"] = df["pca_features"].apply(lambda x: x[1])

df["z"] = df["pca_features"].apply(lambda x: x[2])

df = df[["token", "label", "x", "y", "z"]]

df

Out[38]:

,token,label,x,y,z
0,I,Accounts,9.850468,0.021825,1.712889
1,called,Accounts,0.570325,0.346662,-2.867727
2,Huntington,Accounts,8.635447,0.880233,-0.841711
3,Bank,Accounts,9.391061,0.450668,-1.215742
4,to,Accounts,-2.093784,-1.126185,4.473377
...,...,...,...,...,...
1364,the,Mortgage,0.207832,1.212174,2.345685
1365,company,Mortgage,0.975878,1.152568,1.548877
1366,never,Mortgage,-0.009449,-1.360505,-0.080950
1367,responds,Mortgage,-1.310536,-0.395197,-1.634090


In [0]:
import plotly.express as px

fig = px.scatter_3d(df, x = 'x', y = 'y', z = 'z', color = "label", width=1000, height = 800, hover_data = ["token", "label"])

fig.show()